## 1. Exploration des données
### combien de notes, de films uniques et de user uniques dans le jeu de données ?
### Quelle est la moyenne des notes par user et par film?


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
ratings = pd.read_csv("ratings.csv")
ratings.head()

In [ ]:
movies = pd.read_csv("movies.csv")
movies.head()

In [ ]:
n_ratings = len(ratings)
n_movies = ratings['movieId'].nunique()
n_users = ratings['userId'].nunique()

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average number of ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average number of ratings per movie: {round(n_ratings/n_movies, 2)}")

## Combien de notes chaque user fait-il? Utiliser groupby() et count() pour

      - grouper les données par userId
      - compter le nombre de notes pour chaque userId



In [ ]:
user_freq = ratings[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
user_freq.head()

In [ ]:
print(f"Mean number of ratings for a given user: {user_freq['n_ratings'].mean():.2f}.")


## Visualiser la distribution des notes des films (rating dans le jeu de donnée et la distribution de la fréquence des notes des users

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
ax = sns.countplot(x="rating", data=ratings, palette="viridis")
plt.title("Distribution of movie ratings")

plt.subplot(1,2,2)
ax = sns.kdeplot(user_freq['n_ratings'], shade=True, legend=False)
plt.axvline(user_freq['n_ratings'].mean(), color="k", linestyle="--")
plt.xlabel("# ratings per user")
plt.ylabel("density")
plt.title("Number of movies rated per user")
plt.show()

## Quels sont les films les mieux et les moins bien notés ?

## Pour trouver les « meilleurs » et les « pires » films, vous devez calculer la note moyenne de chaque film de notre ensemble de données. 
## Nous pouvons le faire en regroupant par movieId et en calculant la moyenne de la colonne « rating ».

In [ ]:
mean_rating = ratings.groupby('movieId')[['rating']].mean()
mean_rating.head()



## Quels sont les films les mieux notés et les moins bien notés ?

In [ ]:
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]



In [ ]:
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]

## Qui a noté ce film?

In [ ]:
ratings[ratings['movieId']==highest_rated]

### Le film avec la note moyenne la plus élevée n'a reçu que 2 notes. Ce n'est pas une bonne mesure de la popularité d'un film. La qualité de l'évaluation d'un film dépend non seulement de la note moyenne, mais aussi du nombre d'évaluations.

### Calculons la Moyenne bayésienne
### Nous devons d'abord obtenir le nombre et la moyenne pour chaque film. Nous pouvons le faire avec un groupby qui applique à la fois la moyenne et le nombre à l'aide de la méthode agg.


### ri = (Cx m + somme(ratings))/C+N



In [ ]:
movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

In [ ]:
C = movie_stats['count'].mean()
m = movie_stats['mean'].mean()

def bayesian_avg(ratings):
    bayesian_avg = (C*m+ratings.sum())/(C+ratings.count())
    return bayesian_avg

bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
movie_stats = movie_stats.merge(bayesian_avg_ratings, on='movieId')


##  Quels sont les films avec la moyenne bayésienne  la plus élevée ?




In [ ]:
movie_stats = movie_stats.merge(movies[['movieId', 'title']])
movie_stats.sort_values('bayesian_avg', ascending=False).head()

## En utilisant les moyennes bayésiennes, nous pouvons constater que le film n'est plus en tête du classement.

In [ ]:
movie_stats.sort_values('bayesian_avg', ascending=True).head()

## 2. Transformation des données
### Besoin de transformer les données en matrice utilisateur-item pour le filtrage collaboratif scipy.sparse_matrix : les colonnes sont des films et les lignes des utilisateurs
### Chaque cellule est remplie avec l'évaluation d'un utilisateur par rapport à un film
### Cellule vide = pas d'évaluation disponible

In [ ]:
from scipy.sparse import csr_matrix

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.
    
    Args:
        df: pandas dataframe
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    N = df['userId'].nunique()
    M = df['movieId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
    
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
    
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [ ]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)


## Calculons la Sparsity de la matrice

In [ ]:
sparsity = X.count_nonzero()/(X.shape[0]*X.shape[1])

print(f"Matrix sparsity: {round(sparsity*100,2)}%")


## Pour sauvegarder notre matrice éparse en vue d'une analyse ultérieure, nous pouvons utiliser la méthode save_npz de scipy.sparse

In [ ]:
from scipy.sparse import save_npz

save_npz('user_item_matrix.npz', X)


In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    """
    Finds k-nearest neighbours for a given movie id.
    
    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations
    
    Returns:
        list of k similar movie ID's
    """
    neighbour_ids = []
    
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

## 3. Trouver des films similaires avec k-Nearest Neighbours
## Vous pouvez tester la fonction en lui transmettant un numéro de film (movieId). Créez un dictionnaire qui associe movieId au titre du film afin de mieux interpréter les résultats. Dans ce cas, movie_id = 1 correspond à Toy Story. Nous fixerons k = 10 et utiliserons la métrique par défaut, la similarité cosine. Cela signifie que nous recherchons les 10 films les plus similaires à Toy Story.

In [ ]:
movie_titles = dict(zip(movies['movieId'], movies['title']))

movie_id = 1

similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]

print(f"Because you watched {movie_title}")
for i in similar_ids:
    print(movie_titles[i])
